In [26]:
import numpy as np
import pandas as pd

In [27]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
car_evaluation = fetch_ucirepo(id=19) 
  
# data (as pandas dataframes) 
X = car_evaluation.data.features 
y = car_evaluation.data.targets

In [28]:
from sklearn.model_selection import train_test_split

# data split, 70% training and 30% temp (temp = validation + test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)

# 30% temp data into 15% validation and 15% test
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [41]:
def calculate_metrics(y_true, y_pred):
    # Check if y_true is a pandas DataFrame and convert it to a NumPy array for compatibility
    if isinstance(y_true, pd.DataFrame):
        y_true = y_true.to_numpy()  # Convert y_true to a NumPy array

    # Check if y_pred is a pandas DataFrame and convert it to a NumPy array for compatibility
    if isinstance(y_pred, pd.DataFrame):
        y_pred = y_pred.to_numpy()  # Convert y_pred to a NumPy array

    # Check if y_true is already flattened (1D)
    if y_true.ndim > 1:
        y_true = y_true.flatten()  # Flatten if it has more than 1 dimension

    # Check if y_pred is already flattened (1D)
    if y_pred.ndim > 1:
        y_pred = y_pred.flatten()  # Flatten if it has more than 1 dimension

    # Find unique class names in y_true and determine the number of unique classes
    class_names = np.unique(y_true)  # Get unique class names from y_true
    unique_classes = class_names.size  # Count the number of unique classes

    # Initialize a confusion matrix with zeros, sized based on the number of unique classes
    confusion_matrix = np.zeros((unique_classes, unique_classes), dtype=int)

    # Map class names to indices for easy lookup
    class_name_to_index = {class_name: idx for idx, class_name in enumerate(class_names)}

    # Count occurrences of actual vs predicted labels
    for actual, predicted in zip(y_true, y_pred):
        # Ensure actual and predicted are scalar values, not arrays
        actual = actual.item() if isinstance(actual, np.ndarray) else actual
        predicted = predicted.item() if isinstance(predicted, np.ndarray) else predicted
        
        # Find the index for the actual and predicted class
        actual_index = class_name_to_index[actual]
        predicted_index = class_name_to_index[predicted]

        # Increment the appropriate cell in the confusion matrix
        confusion_matrix[actual_index, predicted_index] += 1

    # Print the confusion matrix row by row
    print("\nConfusion matrix:")
    for row in confusion_matrix:
        print(" ".join(map(str, row)))  # Print each row of the confusion matrix

    # --- --- --- --- --- ---
    
    # Accuracy Calculation
    
    # Sum of the diagonal elements (correct predictions)
    correct_predictions = np.trace(confusion_matrix)  # np.trace() gives the sum of diagonal elements

    # Total number of predictions (sum of all elements in the matrix)
    total_predictions = np.sum(confusion_matrix)

    # Calculate accuracy
    accuracy = correct_predictions / total_predictions
    print(f"\nAccuracy: {accuracy}")

    # --- --- --- --- --- ---
        
    # Precision Calculation

    def calculate_precision(confusion_matrix, class_names):
        precision = {}
        
        # Iterate over each class to calculate its precision
        for i, class_name in enumerate(class_names):
            # True Positive (TP) is the value in the diagonal for that class
            true_positive = confusion_matrix[i, i]
            
            # False Positive (FP) is the sum of the column (excluding the diagonal)
            false_positive = np.sum(confusion_matrix[:, i]) - true_positive
            
            # Precision for the current class
            precision[class_name] = true_positive / (true_positive + false_positive) if (true_positive + false_positive) != 0 else 0        
        
        return precision
    
    precision = calculate_precision(confusion_matrix, class_names)
    print("\nPrecision for each class:")
    for class_name, precision_value in precision.items():
        print(f"{class_name}: {precision_value:}")

    total_precision = sum(precision.values())
    print(f"\nMacro precision: {total_precision / unique_classes}")

    # --- --- --- --- --- ---
    
    # Recall Calculation
    
    def calculate_recall(confusion_matrix, class_names):
        recall = {}
        
        # Iterate over each class to calculate its recall
        for i, class_name in enumerate(class_names):
            # True Positive (TP) is the value in the diagonal for that class
            true_positive = confusion_matrix[i, i]
            
            # False Negative (FN) is the sum of the row (excluding the diagonal)
            false_negative = np.sum(confusion_matrix[i, :]) - true_positive
            
            # Recall for the current class
            recall[class_name] = true_positive / (true_positive + false_negative) if (true_positive + false_negative) != 0 else 0        
        
        return recall
    
    recall = calculate_recall(confusion_matrix, class_names)
    print("\nRecall for each class:")
    for class_name, recall_value in recall.items():
        print(f"{class_name}: {recall_value:.4f}")

    total_recall = sum(recall.values())
    print(f"\nMacro recall: {total_recall / unique_classes}")

    # --- --- --- --- --- ---

    # F1 Score Calculation
    
    def calculate_f1_score(precision, recall):
        f1_scores = {}
    
        # Calculate F1 score for each class
        for class_name in precision.keys():
            p = precision[class_name]
            r = recall[class_name]
            
            # Calculate F1 score for the class, handling cases where p + r = 0
            f1_scores[class_name] = (2 * p * r) / (p + r) if (p + r) != 0 else 0
        
        return f1_scores
    
    f1_scores = calculate_f1_score(precision, recall)
    print("\nF1 Score for each class:")
    for class_name, f1_value in f1_scores.items():
        print(f"{class_name}: {f1_value:}")
        
    # Macro F1 Score Calculation
    total_f1 = sum(f1_scores.values())  # Sum of F1 scores for each class
    macro_f1 = total_f1 / len(f1_scores)  # Average F1 score across all classes
    
    print(f"\nMacro F1 score: {macro_f1:}")

In [30]:
print("Type of X_train:", type(X_train))
print("Shape of X_train:", X_train.shape)
print("Type of y_train:", type(y_train))
print("Shape of y_train:", y_train.shape)
print("Type of X_temp:", type(X_temp))
print("Shape of X_temp:", X_temp.shape)
print("Type of y_temp:", type(y_temp))
print("Shape of y_temp:", y_temp.shape)

Type of X_train: <class 'pandas.core.frame.DataFrame'>
Shape of X_train: (1209, 6)
Type of y_train: <class 'pandas.core.frame.DataFrame'>
Shape of y_train: (1209, 1)
Type of X_temp: <class 'pandas.core.frame.DataFrame'>
Shape of X_temp: (519, 6)
Type of y_temp: <class 'pandas.core.frame.DataFrame'>
Shape of y_temp: (519, 1)


**ZeroR** <br>
In a multiclass classification setting, ZeroR will look at the target values in the training data, count the frequencies of each class, and select the most frequent class as its prediction for all instances, regardless of the input features. <br>
ZeroR doesn't learn from input features, so it only needs the training data to calculate the most frequent class. There's no need to tune hyperparameters, so a validation set isn't required.

**ZeroR Model Implementation**  
This cell implements the ZeroR algorithm, which predicts the most frequent class from the training data, ignoring all feature information. It calculates the mode of the target variable (`y_train`) and uses this most frequent class to make predictions for the training, validation, and test sets. The accuracy of the model is then evaluated by comparing the predicted labels with the actual labels.

In [43]:
import numpy as np
from sklearn.metrics import accuracy_score

# Get the most frequent class from the training set
most_frequent_class = y_train.mode(axis=0).iloc[0]  # Get the most frequent class label

# Predict the most frequent class for all samples in the training, validation, and test sets
y_train_pred = np.full(y_train.shape[0], most_frequent_class)
y_val_pred = np.full(y_val.shape[0], most_frequent_class)
y_test_pred = np.full(y_test.shape[0], most_frequent_class)

# Calculate accuracy using the true labels and the predicted labels
train_accuracy = accuracy_score(y_train, y_train_pred)
val_accuracy = accuracy_score(y_val, y_val_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

# Print ZeroR model accuracy
print("ZeroR Model Accuracy:")
print("Training Accuracy:", train_accuracy)
print("Validation Accuracy:", val_accuracy)
print("Test Accuracy:", test_accuracy)

print("------------------------------")
print("Validation Set:")
calculate_metrics(y_val, y_val_pred)

print("------------------------------")
print("Test Set:")
calculate_metrics(y_test, y_test_pred)

ZeroR Model Accuracy:
Training Accuracy: 0.7047146401985112
Validation Accuracy: 0.6872586872586872
Test Accuracy: 0.6923076923076923
------------------------------
Validation Set:

Confusion matrix:
0 0 57 0
0 0 10 0
0 0 178 0
0 0 14 0

Accuracy: 0.6872586872586872

Precision for each class:
acc: 0
good: 0
unacc: 0.6872586872586872
vgood: 0

Macro precision: 0.1718146718146718

Recall for each class:
acc: 0.0000
good: 0.0000
unacc: 1.0000
vgood: 0.0000

Macro recall: 0.25

F1 Score for each class:
acc: 0
good: 0
unacc: 0.8146453089244851
vgood: 0

Macro F1 score: 0.20366132723112126
------------------------------
Test Set:

Confusion matrix:
0 0 61 0
0 0 9 0
0 0 180 0
0 0 10 0

Accuracy: 0.6923076923076923

Precision for each class:
acc: 0
good: 0
unacc: 0.6923076923076923
vgood: 0

Macro precision: 0.17307692307692307

Recall for each class:
acc: 0.0000
good: 0.0000
unacc: 1.0000
vgood: 0.0000

Macro recall: 0.25

F1 Score for each class:
acc: 0
good: 0
unacc: 0.8181818181818181
vgoo